# Testes de treinamento incremental

## Abordagem antiga

Nos experimentos anteriores, foi testado diferentes train/test splits e como isso influencia no aprendizado dos modelos. Neles, os algoritmos de MAB iam aprendendo incrementalmente na partição de teste. Porém, nesses experimentos, a partição de teste é sempre modificada ao se mudar o train/test split, gerando problemas para analisar os resultados de diferentes splits.

## Abordagem nova

Abordando o problema de uma forma diferente, agora vamos fazer um train/test split inicial. Depois desse split inicial, parte da partição de treino será selecionado para filtrar a partição de teste (removendo cold-start). Por exemplo, podemos escolher começar apenas com 50% da base de treino, então, será considerado apenas as 50% primeiras interações de treino para fazer a filtragem da partição de teste (removendo os usuários e itens que nunca foram vistos nesses 50% da partição de teste).

Agora, com uma base de teste fixa, vamos treinando os modelos usando diferentes tamanhos da base de treino,. Primeiro, é usado o tamanho que foi utilizado para fazer a filtragem do teste (no exemplo foi 50%), e depois, vamos incrementando esse tamanho de pouco em pouco (por exemplo, podemos incrementar de 5 em 5%, ficando 55%, depois 60%, … e assim por diante, até chegar em 100%).

In [1]:
import pandas as pd
from mab2rec import BanditRecommender, LearningPolicy
from mab2rec.pipeline import train
from sklearn.preprocessing import LabelEncoder
import implicit
from scipy.sparse import csr_matrix
from implicit.nearest_neighbours import bm25_weight
import plotly.express as px
import time
import os
import math

train_data = "../data/ml100k/data_train.csv"
test_data = "../data/ml100k/data_test.csv"

In [7]:
FACTORS = 10
K1 = 100
B = 0.8

In [8]:

def train_embeddings_model(Model, df, num_users, num_items, generate_embeddings=False):
    sparse_matrix = csr_matrix((df['response'], (df['user_id'], df['item_id'])), shape=(num_users, num_items))

    model = Model(factors=FACTORS, random_state=1)
    model.fit(sparse_matrix)

    if not generate_embeddings:
        return model, sparse_matrix
    
    user_features_list = []

    for user_id in df['user_id'].unique():
        user_factors = model.user_factors[user_id][:FACTORS]  # O BPR coloca 1 no final dos vetores latentes ?
        user_features_list.append([user_id] + list(user_factors))

    
    df_user_features = pd.DataFrame(user_features_list, columns=['user_id'] + [f'u{i}' for i in range(FACTORS)])

    return model, sparse_matrix, df_user_features

In [9]:

def test_embeddings_model(model, sparse_matrix, df_test):
    all_recs = []

    start_time = time.time()
    hits = 0
    for i, interaction in df_test.iterrows():
        ids_recs, _ = model.recommend(userid=interaction['user_id'], user_items=sparse_matrix[interaction['user_id']], N=10)
        if interaction['item_id'] in ids_recs:
            hits += 1
        all_recs.append(ids_recs.tolist())
    
    recs_df = pd.DataFrame({
        'interaction_number': [i for i in range(len(df_test))],
        'user_id': df_test['user_id'],
        'item_id': df_test['item_id'],
        'recommendations': all_recs
    })
    

    return hits, hits/len(df_test), time.time() - start_time, recs_df

In [10]:

def test_non_incremental(mab_algo, user_features, df_test):
    start_time = time.time()
    hits = 0

    contexts = df_test.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

    recomendations = mab_algo.recommend(contexts)

    df_test = df_test.reset_index(drop=True)

    hits = 0
    for i, interaction in df_test.iterrows():
        if interaction['item_id'] in recomendations[i]:
            hits += 1
    
    recs_df = pd.DataFrame({
        'interaction_number': [i for i in range(len(df_test))],
        'user_id': df_test['user_id'],
        'item_id': df_test['item_id'],
        'recommendations': recomendations
    })

    return hits, hits/len(df_test), time.time() - start_time, recs_df

In [11]:

def test_incremental(mab_algo, user_features, df_test, df_test_for_evaluation, batch_size):
    recs = []

    start_time = time.time()
    hits = 0

    for i in range(0, len(df_test), batch_size):
        # Fazendo recomendações para teste
        df_batch_test = df_test_for_evaluation.loc[i:i+batch_size-1]
        contexts = df_batch_test.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        if len(contexts) > 0: # Se não tiver nenhuma interação positiva, não faz sentido fazer recomendações
            recomendations = mab_algo.recommend(contexts)
            if isinstance(recomendations, list) and isinstance(recomendations[0], int):
                # Quando o contexto tem tamanho 1, a recomendação é uma lista, e não uma lista de listas
                recomendations = [recomendations]

            df_batch_test = df_batch_test.reset_index(drop=True)

            for j, interaction in df_batch_test.iterrows():
                if interaction['item_id'] in recomendations[j]:
                    hits += 1
            
            recs.extend(recomendations)
        
        # Treinando com o batch
        df_batch_train = df_test.loc[i:i+batch_size-1]
        contexts = df_batch_train.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        mab_algo.partial_fit(df_batch_train['item_id'], df_batch_train['response'], contexts)
    
    recs_df = pd.DataFrame({
        'interaction_number': [i for i in range(len(df_test_for_evaluation))],
        'user_id': df_test_for_evaluation['user_id'],
        'item_id': df_test_for_evaluation['item_id'],
        'recommendations': recs
    })

    return hits, hits/len(df_test_for_evaluation), time.time() - start_time, recs_df

In [12]:

def test(test_size, train_initial_size, train_increment_step_size):
    '''
    - `test_size`: define o tamanho da partição de teste no train/test split inicial. Por exemplo, se for escolhido 0.1 (10%), a partição de teste terá 10% das interações e a partição de treino terá 90% das interações. O tamanho da partição de teste passará ainda por um filtro com o tamanho do treino inicial, definido no próximo parâmetro.
    - `train_initial_size`: define o tamanho inicial que será usado para treino dos modelos. Esse tamanho é uma porcentagem da partição de treino, por exemplo, 0.5 (50%) quer dizer que o treino será feito inicialmente com 50% das interações separadas para treino. Vale ressaltar que essa porcentagem é relacionada apenas à partição de treino, então, se temos uma partição de treino de 0.9 (90%) e o “train_initial_size” é definido como 0.5 (50%), então, teremos 45% (0.9 * 0.5) das interações todas para o treino inicial. Com a base de treino separada com essa porcentagem inicial, a base de teste passara por um filtro, removendo todas as interações com itens ou usuários que nunca foram vistos nesse treino inicial.
    - `train_increment_step_size`: define de quanto em quanto a base de treino irá crescer para o treino dos modelos. Esse incremento será feito sobre o tamanho inicial de treino. Por exemplo, se temos “train_initial_size” como 0.5 (50%), e temos “train_increment_step_size” como 0.05 (5%), então, iremos incrementar para 55%, depois para 60%, … e assim por diante até chegar em 100%.
    '''
    results = []
    df_recs = pd.DataFrame(columns=['algorithm', 'interaction_number', 'user_id', 'item_id', 'recommendations'])
    df_train = pd.read_csv(train_data)
    df_test = pd.read_csv(test_data)

    df_full = pd.concat([df_train, df_test])

    df_full['user_id'] = LabelEncoder().fit_transform(df_full['user_id'])
    df_full['item_id'] = LabelEncoder().fit_transform(df_full['item_id'])

    num_users = df_full['user_id'].nunique()
    num_items = df_full['item_id'].nunique()

    split_index = int(len(df_full) * (1 - test_size))
    df_train_full = df_full[:split_index]
    df_test = df_full[split_index:]

    current_df_train = df_train_full[:int(len(df_train_full) * train_initial_size)]

    df_test = df_test[(df_test['user_id'].isin(current_df_train['user_id'])) & (df_test['item_id'].isin(current_df_train['item_id']))]
    df_test = df_test.reset_index(drop=True)
    df_test_for_evaluation = df_test[df_test['response'] == 1]

    print('Generating ALS embeddings')
    ALS_model, _, ALS_user_features = train_embeddings_model(implicit.als.AlternatingLeastSquares, current_df_train, num_users, num_items, generate_embeddings=True)

    print('Generating BPR embeddings')
    BPR_model, _, BPR_user_features = train_embeddings_model(implicit.bpr.BayesianPersonalizedRanking, current_df_train, num_users, num_items, generate_embeddings=True)

    current_train_size = train_initial_size
    while current_train_size <= 1:
        print(f"Current train size: {current_train_size}")

        current_df_train = df_train_full[:int(len(df_train_full) * current_train_size)]

        # -------------- ALS -----------------
        print('Training ALS')
        ALS_model, sparse_matrix = train_embeddings_model(implicit.als.AlternatingLeastSquares, current_df_train, num_users, num_items)

        print('Testing ALS')
        hits, hr, spent_time, df_recs_als = test_embeddings_model(ALS_model, sparse_matrix, df_test_for_evaluation)
        df_recs_als['algorithm'] = 'ALS'
        df_recs_als['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_als])
        results.append({'algorithm': 'ALS', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})


        # -------------- BPR -----------------
        print('Training BPR')
        BPR_model, sparse_matrix = train_embeddings_model(implicit.bpr.BayesianPersonalizedRanking, current_df_train, num_users, num_items)

        print('Testing BPR')
        hits, hr, spent_time, df_recs_bpr = test_embeddings_model(BPR_model, sparse_matrix, df_test_for_evaluation)
        df_recs_bpr['algorithm'] = 'BPR'
        df_recs_bpr['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_bpr])
        results.append({'algorithm': 'BPR', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})


        # ------ LinUCB - non-incremental - ALS embeddings -------
        print('Training LinUCB - non-incremental - ALS embeddings')
        linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(alpha=0.1), top_k=10)
        train(linUCB_model, data=current_df_train, user_features=ALS_user_features)

        print('Testing LinUCB - non-incremental - ALS embeddings')
        hits, hr, spent_time, df_recs_linUCB = test_non_incremental(linUCB_model, ALS_user_features, df_test_for_evaluation)
        df_recs_linUCB['algorithm'] = 'LinUCB - non-incremental - ALS embeddings'
        df_recs_linUCB['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_linUCB])
        results.append({'algorithm': 'LinUCB - non-incremental - ALS embeddings', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})


        # ------ LinUCB - non-incremental - BPR embeddings -------
        print('Training LinUCB - non-incremental - BPR embeddings')
        linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(alpha=0.1), top_k=10)
        train(linUCB_model, data=current_df_train, user_features=BPR_user_features)

        print('Testing LinUCB - non-incremental - BPR embeddings')
        hits, hr, spent_time, df_recs_linUCB = test_non_incremental(linUCB_model, BPR_user_features, df_test_for_evaluation)
        df_recs_linUCB['algorithm'] = 'LinUCB - non-incremental - BPR embeddings'
        df_recs_linUCB['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_linUCB])
        results.append({'algorithm': 'LinUCB - non-incremental - BPR embeddings', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})


        # ------ LinUCB - incremental - ALS embeddings -------
        print('Training LinUCB - incremental - ALS embeddings')
        linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(alpha=0.1), top_k=10)
        train(linUCB_model, data=current_df_train, user_features=ALS_user_features)

        print('Testing LinUCB - incremental - ALS embeddings')
        hits, hr, spent_time, df_recs_linUCB = test_incremental(linUCB_model, ALS_user_features, df_test, df_test_for_evaluation, batch_size=10)
        df_recs_linUCB['algorithm'] = 'LinUCB - incremental - ALS embeddings'
        df_recs_linUCB['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_linUCB])
        results.append({'algorithm': 'LinUCB - incremental - ALS embeddings', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})


        # ------ LinUCB - incremental - BPR embeddings -------
        print('Training LinUCB - incremental - BPR embeddings')
        linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(alpha=0.1), top_k=10)
        train(linUCB_model, data=current_df_train, user_features=BPR_user_features)

        print('Testing LinUCB - incremental - BPR embeddings')
        hits, hr, spent_time, df_recs_linUCB = test_incremental(linUCB_model, BPR_user_features, df_test, df_test_for_evaluation, batch_size=10)
        df_recs_linUCB['algorithm'] = 'LinUCB - incremental - BPR embeddings'
        df_recs_linUCB['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_linUCB])
        results.append({'algorithm': 'LinUCB - incremental - BPR embeddings', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})


        # ------ LinGreedy - non-incremental - ALS embeddings -------
        print('Training LinGreedy - non-incremental - ALS embeddings')
        linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(epsilon=0.01), top_k=10)
        train(linGreedy_model, data=current_df_train, user_features=ALS_user_features)

        print('Testing LinGreedy - non-incremental - ALS embeddings')
        hits, hr, spent_time, df_recs_linGreedy = test_non_incremental(linGreedy_model, ALS_user_features, df_test_for_evaluation)
        df_recs_linGreedy['algorithm'] = 'LinGreedy - non-incremental - ALS embeddings'
        df_recs_linGreedy['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_linGreedy])
        results.append({'algorithm': 'LinGreedy - non-incremental - ALS embeddings', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})


        # ------ LinGreedy - non-incremental - BPR embeddings -------
        print('Training LinGreedy - non-incremental - BPR embeddings')
        linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(epsilon=0.01), top_k=10)
        train(linGreedy_model, data=current_df_train, user_features=BPR_user_features)

        print('Testing LinGreedy - non-incremental - BPR embeddings')
        hits, hr, spent_time, df_recs_linGreedy = test_non_incremental(linGreedy_model, BPR_user_features, df_test_for_evaluation)
        df_recs_linGreedy['algorithm'] = 'LinGreedy - non-incremental - BPR embeddings'
        df_recs_linGreedy['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_linGreedy])
        results.append({'algorithm': 'LinGreedy - non-incremental - BPR embeddings', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})


        # ------ LinGreedy - incremental - ALS embeddings -------
        print('Training LinGreedy - incremental - ALS embeddings')
        linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(epsilon=0.01), top_k=10)
        train(linGreedy_model, data=current_df_train, user_features=ALS_user_features)

        print('Testing LinGreedy - incremental - ALS embeddings')
        hits, hr, spent_time, df_recs_linGreedy = test_incremental(linGreedy_model, ALS_user_features, df_test, df_test_for_evaluation, batch_size=10)
        df_recs_linGreedy['algorithm'] = 'LinGreedy - incremental - ALS embeddings'
        df_recs_linGreedy['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_linGreedy])
        results.append({'algorithm': 'LinGreedy - incremental - ALS embeddings', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})


        # ------ LinGreedy - incremental - BPR embeddings -------
        print('Training LinGreedy - incremental - BPR embeddings')
        linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(epsilon=0.01), top_k=10)
        train(linGreedy_model, data=current_df_train, user_features=BPR_user_features)

        print('Testing LinGreedy - incremental - BPR embeddings')
        hits, hr, spent_time, df_recs_linGreedy = test_incremental(linGreedy_model, BPR_user_features, df_test, df_test_for_evaluation, batch_size=10)
        df_recs_linGreedy['algorithm'] = 'LinGreedy - incremental - BPR embeddings'
        df_recs_linGreedy['train_size'] = current_train_size
        df_recs = pd.concat([df_recs, df_recs_linGreedy])
        results.append({'algorithm': 'LinGreedy - incremental - BPR embeddings', 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_train_size})
        
        # Incrementando o tamanho do treino para próxima iteração
        current_train_size = round(current_train_size + train_increment_step_size, 2)
    
    save_path = f'results-v4/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    df_results = pd.DataFrame(results)
    df_results = df_results.astype({'hits': int, 'hr': float, 'time': float})
    df_results['test_size'] = round(test_size, 2)
    df_results['test_interactions'] = len(df_test_for_evaluation)

    df_results.to_csv(f'{save_path}/results.csv', index=False)
    df_recs.to_csv(f'{save_path}/recs.csv', index=False)

In [ ]:
test(test_size=0.1, train_initial_size=0.5, train_increment_step_size=0.05)

In [2]:
df_results = pd.read_csv('results-v4/results.csv')

In [3]:
fig = px.line(df_results, x="train_size", y="hr", color='algorithm', title='HR x Train size')
fig.show()
fig.write_html('results-v4/hr_x_train_size.html')